install https://github.com/cjer/bclm  
install https://github.com/EmilStenstrom/conllu

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np

In [4]:
import bclm

In [5]:
dropped = [5438, 5444, 5445, 5446, 5448, 5449, 5450, 5451, 5453, 5459]
spdf = bclm.read_dataframe('spmrl')

In [6]:
global_dropped = [spdf[spdf.sent_id==d].global_sent_id.iat[0] for d in dropped]
global_dropped

[5438, 5444, 5445, 5446, 5448, 5449, 5450, 5451, 5453, 5459]

In [7]:
uddf = bclm.read_dataframe('ud')

In [8]:
uddf = uddf[(~uddf.global_sent_id.isin(global_dropped))]

In [70]:
spdf = spdf[(~spdf.global_sent_id.isin(global_dropped))]

In [9]:
from collections import OrderedDict

In [11]:
ud_sents = list(uddf.sent_id.unique())

In [15]:
def get_set_from_sent_id(gsi):
    if gsi>=1 and gsi<=484:
        return 'dev'
    elif gsi>=485 and gsi<=5725:
        return 'train'
    elif gsi>=5726 and gsi<=6216:
        return 'test'
    
uddf['ud_set'] = uddf.sent_id.apply(get_set_from_sent_id)

uddf.groupby(['ud_set']).sent_id.agg(['min', 'max'])

,min,max
ud_set,,
dev,1,484
test,5726,6216
train,485,5725


In [19]:
uddf['set'] = uddf['ud_set']

,sent_id,token_id,token_str,biose_layer0,upostag,set
0,1,1,עשרות,O,NUM,dev
1,1,2,אנשים,O,NOUN,dev
2,1,3,מגיעים,O,VERB,dev
3,1,4,מתאילנד,O^S-GPE,ADP^PROPN,dev
4,1,5,לישראל,O^S-GPE,ADP^PROPN,dev


## TODO:
1. Add text
1. calculate roots and edges
1. Calculate NER start/stop positions
1. Add NER for all layers
1. Create dict from df sentence
1. Write jsonl
1. NOT NOW: add offsets

### Add text

In [29]:
from conllu import parse 
with open('align/ud_fixed.conllu', 'r', encoding='utf8') as f:
    ud_conllu_sents = parse(f.read())

In [35]:
ud_conllu_sents[1].metadata

OrderedDict([('sent_id', '2'),
             ('global_sent_id', '2'),
             ('text',
              'תופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת, שדנה בנושא העסקת עובדים זרים.'),
             ('very_similar_sent_id', '{}'),
             ('duplicate_sent_id', '{}'),
             ('tokens',
              'תופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת , שדנה בנושא העסקת עובדים זרים .'),
             ('set', 'dev')])

In [33]:
ud_texts = {int(s.metadata['sent_id']): s.metadata['text'] for s in ud_conllu_sents}
len(ud_texts)

6153

In [34]:
uddf['text'] = uddf.sent_id.map(ud_texts)
uddf.head().T

,0,1,2,3,4
sent_id,1,1,1,1,1
id,1,2,3,4,5
form,עשרות,אנשים,מגיעים,מ,תאילנד
lemma,עשרות,איש,הגיע,מ,תאילנד
upostag,NUM,NOUN,VERB,ADP,PROPN
xpostag,NUM,NOUN,VERB,ADP,PROPN
feats,"OrderedDict([('Definite', 'Cons'), ('Gender', ...","OrderedDict([('Gender', 'Masc'), ('Number', 'P...","OrderedDict([('Gender', 'Masc'), ('HebBinyan',...",NaN,NaN
head,2,3,0,5,3
deprel,nummod,nsubj,root,case,obl
deps,NaN,NaN,NaN,NaN,NaN


In [65]:
from conllu import parse 
with open('align/spmrl_fixed.conllu', 'r', encoding='utf8') as f:
    sp_conllu_sents = parse(f.read())

In [67]:
sp_conllu_sents[1].metadata

OrderedDict([('sent_id', '2'),
             ('global_sent_id', '2'),
             ('text_from_ud',
              'תופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת, שדנה בנושא העסקת עובדים זרים.'),
             ('very_similar_sent_id', '{}'),
             ('duplicate_sent_id', '{}'),
             ('set', 'dev')])

In [69]:
sp_texts = {int(s.metadata['sent_id']): s.metadata['text_from_ud'] for s in sp_conllu_sents}
len(sp_texts)

6153

In [71]:
spdf['text'] = spdf.sent_id.map(sp_texts)
spdf.head().T

,0,1,2,3,4
id,1,2,3,4,5
form,עשרות,אנשים,מגיעים,מ,תאילנד
lemma,עשר,איש,הגיע,מ,תאילנד
upostag,CDT,NN,BN,PREPOSITION,NNP
xpostag,CDT,NN,BN,PREPOSITION,NNP
feats,gen=F|num=P,gen=M|num=P,gen=M|num=P|per=A|HebBinyan=HIFIL,_,_
token_id,1,2,3,4,4
sent_id,1,1,1,1,1
token_str,עשרות,אנשים,מגיעים,מתאילנד,מתאילנד
global_sent_id,1,1,1,1,1


In [80]:
def get_edges(df):
    return (df
            .rename(columns={'head': 'source', 
                             'id': 'destination', 
                             'deprel': 'relation'})[['source', 'destination', 'relation']]
            .to_dict('records'))

def get_roots(df):
    return [int(x) for x in list(df[(df['head']==0) | (df.deprel.str.lower()=='root')].id.unique())]

def get_overlapping_entities(df, col='biose_layer0', priority=0):
    mentions = []
    current_mention= None
    for i, r in df.iterrows():
        bio = r[col][0]
        if bio!='O':
            cat = r[col].split('-')[1]
        else:
            cat = '-'
        if bio=='S':
            mentions.append({'first': int(r['id']),
                             'last': int(r['id']),
                             'label': cat, 
                             'priority': priority})
            current_mention= None
        if bio=='B':
            current_mention = {'first': int(r['id']), 
                               'last': int(r['id']), 
                               'label': cat,
                               'priority': priority}

#         if bio=='I' and current_mention is not None:
#             pass
        
        if bio=='E' and current_mention is not None:
            current_mention['last'] = int(r['id'])
            mentions.append(current_mention)
            current_mention= None
        if bio=='O':
            current_mention = None
    return mentions
    
def get_sentence_dict(df):
    df = df.sort_values('id')
    d = {}
    d['id'] = int(df.sent_id.iat[0])
    d['text'] = df.text.iat[0]
    s = {}
    s['words'] = df.form.tolist()
    s['raw'] = s['words']
    s['startOffsets'] = []
    s['endOffsets'] = []
    s['tags'] = df.upostag.tolist()
    s['lemmas'] = df.lemma.tolist()
    s['chunks'] = []
    edges = get_edges(df)
    roots = get_roots(df)
    universal_basic = {'edges': edges, 'roots': roots}
    s['graphs'] = {'universal-basic': universal_basic,
                   'universal_enhanced': {'edges': [], 'roots': []}}
    s['metadata'] =  {
                'sentParagraphOffsets': []
            },
    ents = (get_overlapping_entities(df, 'biose_layer0', 0) 
            + get_overlapping_entities(df, 'biose_layer1', 1) 
            + get_overlapping_entities(df, 'biose_layer2', 2) 
            + get_overlapping_entities(df, 'biose_layer3', 3))
    s['overlappingEntities'] = ents
    d['sentences'] = [s,]
    d['metadata'] = {'paragraph_': d['text'], 
                     'section': df.set.iat[0]}
    d['gold_relations']= [],
    d['gold_events']= [],
    d['corefChains']= {'rawMentions': []}
    return d

In [81]:
get_sentence_dict(uddf[uddf.sent_id==1])
    
    

{'id': 1,
 'text': 'עשרות אנשים מגיעים מתאילנד לישראל כשהם נרשמים כמתנדבים, אך למעשה משמשים עובדים שכירים זולים.',
 'sentences': [{'words': ['עשרות',
    'אנשים',
    'מגיעים',
    'מ',
    'תאילנד',
    'ל',
    'ישראל',
    'כש',
    'הם',
    'נרשמים',
    'כ',
    'מתנדבים',
    ',',
    'אך',
    'למעשה',
    'משמשים',
    'עובדים',
    'שכירים',
    'זולים',
    '.'],
   'raw': ['עשרות',
    'אנשים',
    'מגיעים',
    'מ',
    'תאילנד',
    'ל',
    'ישראל',
    'כש',
    'הם',
    'נרשמים',
    'כ',
    'מתנדבים',
    ',',
    'אך',
    'למעשה',
    'משמשים',
    'עובדים',
    'שכירים',
    'זולים',
    '.'],
   'startOffsets': [],
   'endOffsets': [],
   'tags': ['NUM',
    'NOUN',
    'VERB',
    'ADP',
    'PROPN',
    'ADP',
    'PROPN',
    'SCONJ',
    'PRON',
    'VERB',
    'ADP',
    'NOUN',
    'PUNCT',
    'CCONJ',
    'ADV',
    'VERB',
    'NOUN',
    'ADJ',
    'ADJ',
    'PUNCT'],
   'lemmas': ['עשרות',
    'איש',
    'הגיע',
    'מ',
    'תאילנד',
    'ל',
    'י

In [82]:
get_sentence_dict(spdf[spdf.sent_id==1])
    
    

{'id': 1,
 'text': 'עשרות אנשים מגיעים מתאילנד לישראל כשהם נרשמים כמתנדבים, אך למעשה משמשים עובדים שכירים זולים.',
 'sentences': [{'words': ['עשרות',
    'אנשים',
    'מגיעים',
    'מ',
    'תאילנד',
    'ל',
    'ישראל',
    'כש',
    'הם',
    'נרשמים',
    'כ',
    'מתנדבים',
    ',',
    'אך',
    'למעשה',
    'משמשים',
    'עובדים',
    'שכירים',
    'זולים',
    '.'],
   'raw': ['עשרות',
    'אנשים',
    'מגיעים',
    'מ',
    'תאילנד',
    'ל',
    'ישראל',
    'כש',
    'הם',
    'נרשמים',
    'כ',
    'מתנדבים',
    ',',
    'אך',
    'למעשה',
    'משמשים',
    'עובדים',
    'שכירים',
    'זולים',
    '.'],
   'startOffsets': [],
   'endOffsets': [],
   'tags': ['CDT',
    'NN',
    'BN',
    'PREPOSITION',
    'NNP',
    'PREPOSITION',
    'NNP',
    'TEMP',
    'PRP',
    'BN',
    'PREPOSITION',
    'NN',
    'yyCM',
    'CC',
    'RB',
    'BN',
    'NN',
    'JJ',
    'JJ',
    'yyDOT'],
   'lemmas': ['עשר',
    'איש',
    'הגיע',
    'מ',
    'תאילנד',
    'ל',
    'ישרא

In [61]:
all_dicts = [get_sentence_dict(uddf[uddf.sent_id==i]) for i in sorted(list(uddf.sent_id.unique()))]

In [64]:
import jsonlines

with jsonlines.open('data/hebtb_ud.ann.jsonl', mode='w') as writer:
    for d in all_dicts:
        writer.write(d)

In [74]:
sp_all_dicts = [get_sentence_dict(spdf[spdf.sent_id==i]) for i in sorted(list(spdf.sent_id.unique()))]

In [75]:
import jsonlines

with jsonlines.open('data/hebtb_spmrl.ann.jsonl', mode='w') as writer:
    for d in sp_all_dicts:
        writer.write(d)